In [4]:
import h5py
import numpy as np
from matplotlib import pyplot as plt
import csv
from numpy import genfromtxt
import os, os.path
from collections import Counter
from tqdm import tqdm

In [6]:
scene_name = "ai_001_001"

def import_scene(scene_name):
    downloads_dir = r".\contrib\99991\downloads"
    bb_position_path = os.path.join(downloads_dir, scene_name, "_detail\mesh\metadata_semantic_instance_bounding_box_object_aligned_2d_positions.hdf5")

    with h5py.File(bb_position_path, "r") as f: bb_positions = f['dataset'][:]

    evermotion_scene_path = r".\evermotion_dataset\scenes"
    evermotion_path = r".\evermotion_dataset"
    mesh_objects_si_path = os.path.join(evermotion_scene_path, scene_name, r"_detail\mesh\mesh_objects_si.hdf5")
    mesh_objects_sii_path = os.path.join(evermotion_scene_path, scene_name, r"_detail\mesh\mesh_objects_sii.hdf5")
    metadata_objects_path = os.path.join(evermotion_scene_path, scene_name, r"_detail\mesh\metadata_objects.csv")
    nyu_labels_path = r".\code\cpp\tools\scene_annotation_tool\semantic_label_descs.csv"
    scene_info_path = os.path.join(evermotion_path, "analysis", "metadata_camera_trajectories.csv")

    with h5py.File(mesh_objects_si_path, "r") as f: mesh_objects_si = f['dataset'][:]
    with h5py.File(mesh_objects_sii_path, "r") as f: mesh_objects_sii = f['dataset'][:]
    metadata_objects = genfromtxt(metadata_objects_path, delimiter=None, dtype=str)
    nyu_labels = genfromtxt(nyu_labels_path, delimiter=',', dtype=None, encoding=None, autostrip=True)
    scene_info = genfromtxt(scene_info_path, delimiter=',', dtype=str)

    return mesh_objects_si, mesh_objects_sii, metadata_objects, nyu_labels, scene_info, bb_positions

# mesh_objects_si, mesh_objects_sii, metadata_objects, nyu_labels, scene_info, bb_positions = import_scene(scene_name)

In [17]:
def generate_labels(bb_positions, mesh_objects_sii, mesh_objects_si, metadata_objects, nyu_labels):
    n_bb = bb_positions.shape[0] - 1
    bb_labels = []
    no_nyu = []
    bb_error = []

    for i in range(n_bb):
        instances = np.where(mesh_objects_sii == i+1)[0]
        
        if instances.size > 0:
            id = mesh_objects_si[instances[0]]
            if id == 40 or id == 39 or id == 38 or id == -1: 
                low_lvl_name = metadata_objects[instances[-1]]
                low_lvl_name_clean = ''.join(i for i in low_lvl_name.split('_obj_')[0] if not(i.isdigit())).lower()
                if low_lvl_name_clean[-1] == '_':
                    low_lvl_name_clean = low_lvl_name_clean.split('_')[0]
                bb_labels.append(low_lvl_name_clean)
                no_nyu.append(i+1)
            else:
                bb_labels.append(nyu_labels[int(id)][1].upper())
        else:
            bb_error.append(i+1)
            bb_labels.append(np.nan)
    # bb_labels
    return bb_labels
        

In [24]:
DIR = './contrib/99991/downloads'
total_count = {}
for scene in tqdm(os.listdir(DIR)):
    mesh_objects_si, mesh_objects_sii, metadata_objects, nyu_labels, scene_info, bb_positions = import_scene(scene)
    bb_labels = generate_labels(bb_positions, mesh_objects_sii, mesh_objects_si, metadata_objects, nyu_labels)
    label_freq = dict(Counter(bb_labels))
    for key, value in label_freq.items():
        total_count[key] = total_count.setdefault(key, 0) + value

total_count = dict(sorted(total_count.items(), key=lambda item: item[1], reverse=True)) # sort dict
total_count

100%|██████████| 258/258 [00:04<00:00, 54.53it/s]


{'archinteriors': 3948,
 'BOOKS': 3230,
 nan: 2342,
 'CHAIR': 1651,
 'ai': 1353,
 'LAMP': 1148,
 'arch': 903,
 'TABLE': 423,
 'CLOTHES': 403,
 'PILLOW': 356,
 'WINDOW': 354,
 'BOX': 333,
 'detail': 291,
 'CABINET': 216,
 'BLINDS': 212,
 'PICTURE': 201,
 'object': 128,
 'equipment': 128,
 'SOFA': 108,
 'DOOR': 106,
 'COUNTER': 82,
 'BOOKSHELF': 80,
 'FLOORMAT': 74,
 'SINK': 67,
 'TOWEL': 62,
 'SHELVES': 62,
 'DESK': 62,
 'MIRROR': 50,
 'glass': 44,
 'orange': 44,
 'vrayproxy': 43,
 'TELEVISION': 42,
 'table': 41,
 'archmodels': 40,
 'equp': 32,
 'chess': 32,
 'plant': 30,
 'line': 30,
 'apple': 28,
 'decoration': 25,
 'BED': 25,
 'kitchen_stuff': 25,
 'lemon': 23,
 'objarch': 23,
 'shelf_stuff': 22,
 'archinteriros': 22,
 'wine': 21,
 'PAPER': 21,
 'NIGHTSTAND': 19,
 'dinner_set': 19,
 'BATHTUB': 17,
 'obj': 17,
 'cookie': 16,
 'plate': 16,
 'CURTAIN': 16,
 'box': 15,
 'cylinder': 15,
 'korek': 15,
 'halogen': 14,
 'ai__cylinder': 14,
 'TOILET': 13,
 'speaker': 13,
 'condiments': 13,
 '